In [88]:
import numpy as np
import pandas as pd
import json
import gzip
import os
import datetime
from dateutil import parser
from tqdm import tqdm

import pandas as pd
import numpy as np
import pickle
import datetime
import json
import re
import os, gzip, shutil, fnmatch

import matplotlib
from zipfile import BadZipfile
from matplotlib import pyplot as plt 

In [89]:
## Iterative decompression
from gzip import BadGzipFile

def gz_extract(directory):
    extension = ".gz"
    os.chdir(directory)
    for item in os.listdir(directory): # loop through items in dir
      if item.endswith(extension): # check for ".gz" extension
          gz_name = os.path.abspath(item) # get full path of files
          file_name = (os.path.basename(gz_name)).rsplit('.',1)[0] #get file name for file within
          try:
            with gzip.open(gz_name,"rb") as f_in, open(file_name,"wb") as f_out:
              # print(gz_name)
              shutil.copyfileobj(f_in, f_out)
            os.remove(gz_name) # delete zipped file
          except BadZipfile:
            continue
          except BadGzipFile:
            continue

In [90]:
deviceUsagePath = "/Users/farhan/Desktop/DNL/Budding_Scholar_22-23/ExtractedData/run1_sk_deviceUsage.csv"
visitsPath = "/Users/farhan/Desktop/DNL/Budding_Scholar_22-23/ExtractedData/run1_sk_visits.csv"
outputDirectory = "/Users/farhan/Desktop/DNL/Budding_Scholar_22-23/output_tables"

In [91]:
directory = "/Users/farhan/Desktop/DNL/Budding_Scholar_22-23/Data/"
metric_folder = "sensorkit-visits/iPhone"

In [92]:
from json import JSONDecodeError
visitsList = []

## Loop over all data data folders
for folder in os.listdir(directory):
    path = directory + folder + "/" + metric_folder
    
    if folder == ".DS_Store":
        continue
    
    ## Loop over all participants
    for participant in os.listdir(path):
        ParticipantIdentifier = participant
        pFolder = path + "/" + participant
    
        for data_folder in os.listdir(pFolder):
            
            final_path = pFolder + "/" + data_folder
            
            gz_extract(final_path)

            ## print(path)
            ## Loop over all files in this path/directory
            for fname in os.listdir(final_path):
                filename = ""

                ## name of the file
                if fname.endswith("json"):
                    filename = final_path + "/" + fname
                else: 
                    continue
                
                ## Load the JSON File
                visitsFile = open(filename)
                
                ## Need to use json.load and not json.loads
                ## Load the visits file

                loaded_file = ""
                try:
                    loaded_file = json.load(visitsFile)
                except JSONDecodeError:
                    continue

                ## Get the samples list
                samples = loaded_file["samples"]

                ## Get the name
                name = loaded_file["device"]["name"]

                ## Need a loop here to iterate over all samples
                for i in range(len(samples)):
                    
                    ## Get the TimeStamp for the current sample
                    ParticipantIdentifier = participant
                    dt = parser.parse(samples[i]["timestamp"])
                    trial_date = (dt + datetime.timedelta(hours = -4)).date()
                    time = datetime.time()

                    ## Get the sample dictionary
                    sample_dict_iterator = samples[i]["sample"]

                    ## For the current sample, get the location category
                    locationCategory = sample_dict_iterator["locationCategory"];

                    ## Set up the accumulators
                    visits_gym_elapsedTime = "NaN"; visits_gym_startTime = "NaN"; visits_gym_distance = "NaN"
                    visits_home_elapsedTime = "NaN"; visits_home_startTime = "NaN"; visits_home_distance = "NaN"
                    visits_school_startTime = "NaN"; visits_school_elapsedTime = "NaN"; visits_school_distance = "NaN"
                    visits_unknown_startTime = "NaN"; visits_unknown_elapsedTime = "NaN"; visits_unknown_distance = "NaN"
                    visits_work_startTime = "NaN"; visits_work_elapsedTime = "NaN"; visits_work_distance = "NaN"

                    if locationCategory == "Home":
                        visits_home_startTime = parser.parse(sample_dict_iterator["arrivalDateInterval"]["start"])
                        visits_home_elapsedTime = (parser.parse(sample_dict_iterator['departureDateInterval']['start']) - visits_home_startTime).total_seconds()            
                        visits_home_distance = sample_dict_iterator["distanceFromHome"]

                    elif locationCategory == "Work":
                        visits_work_startTime = parser.parse(sample_dict_iterator["arrivalDateInterval"]["start"])
                        visits_work_elapsedTime = (parser.parse(sample_dict_iterator['departureDateInterval']['start']) - visits_work_startTime).total_seconds() 
                        visits_work_distance = sample_dict_iterator["distanceFromHome"]

                    elif locationCategory == "Gym":
                        visits_gym_startTime = parser.parse(sample_dict_iterator["arrivalDateInterval"]["start"])
                        visits_gym_elapsedTime = (parser.parse(sample_dict_iterator['departureDateInterval']['start']) - visits_gym_startTime).total_seconds() 
                        visits_gym_distance = sample_dict_iterator["distanceFromHome"]

                    elif locationCategory == "School":
                        visits_school_startTime = parser.parse(sample_dict_iterator["arrivalDateInterval"]["start"])
                        visits_school_elapsedTime = (parser.parse(sample_dict_iterator['departureDateInterval']['start']) - visits_school_startTime).total_seconds() 
                        visits_school_distance = sample_dict_iterator["distanceFromHome"]

                    elif locationCategory == "Unknown":
                        visits_unknown_startTime = parser.parse(sample_dict_iterator["arrivalDateInterval"]["start"])
                        visits_unknown_elapsedTime = (parser.parse(sample_dict_iterator['departureDateInterval']['start']) - visits_unknown_startTime).total_seconds() 
                        visits_unknown_distance = sample_dict_iterator["distanceFromHome"]

                    tempVisitsDict = {
                        # 'ParticipantIdentifier': ParticipantIdentifier, 
                        'ParticipantIdentifier': ParticipantIdentifier, 'datetime': dt, 'trial_date': trial_date, 'time': time,
                        'visits_gym_elapsedTime': visits_gym_elapsedTime, 'visits_gym_startTime': visits_gym_startTime, 'visits_gym_distance': visits_gym_startTime,
                        'visits_home_elapsedTime': visits_home_elapsedTime, 'visits_home_startTime': visits_home_startTime, 'visits_home_distance': visits_home_distance,
                        'visits_school_startTime': visits_school_startTime, 'visits_school_elapsedTime': visits_school_elapsedTime, 'visits_school_distance': visits_school_distance,
                        'visits_unknown_startTime': visits_unknown_startTime, 'visits_unknown_elapsedTime': visits_unknown_elapsedTime, 'visits_unknown_distance': visits_unknown_distance,
                        'visits_work_startTime': visits_work_startTime, 'visits_work_elapsedTime': visits_work_elapsedTime, 'visits_work_distance': visits_work_distance
                        }
                    
                    visitsList.append(tempVisitsDict)

In [93]:
visitsDF = pd.DataFrame(visitsList)
visitsDF.to_csv(outputDirectory + '/visitsFarhan.csv', index=False)
# visitsDF.head(5)

/var/folders/57/9mrlx9kx6xsfb5tsnbtdb9d40000gn/T/ipykernel_45701/4229528753.py:1: FutureWarning: Inferring datetime64[ns, tzoffset(None, -14400)] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, tzoffset(None, -14400)])
  visitsDF = pd.DataFrame(visitsList)
